### make sure to install
```
pip install ipympl
pip install nodejs
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install jupyter-matplotlib
```

credit: https://stackoverflow.com/a/56416229/11756613

In [ ]:
%load_ext lab_black
%matplotlib widget
# %matplotlib nbagg should support double clicks, but doesn't work in jupyter :(

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

plt.style.use("dark_background")

In [ ]:
# temporary autocompletion bug fix
%config Completer.use_jedi = False

In [ ]:
def poly(sides, G, n1, n2):
    ns = [n1, n2]
    poss = []
    poss.append(G.nodes[n1]["pos"])
    poss.append(G.nodes[n2]["pos"])
    direction = poss[1] - poss[0]
    assert np.isclose(1, abs(direction), atol=0.2)

    rotation = np.exp(2 * np.pi * 1j / sides)
    for i in range(1, sides - 1):
        poss.append(poss[-1] + direction * rotation ** i)

    existing_poss = nx.get_node_attributes(G, "pos")

    for pos in poss[2:]:
        match = [
            node
            for node, e_pos in existing_poss.items()
            if np.isclose(e_pos, pos, atol=0.2)
        ]
        if match:
            # node already exists in this position, so use it
            assert len(match) == 1
            ns.append(match[0])
        else:
            # no node exists in this position, so create it
            ns.append(len(G.nodes))  # assumes nodes are numbered 0,1,2,...
            G.add_node(ns[-1], pos=pos)

    for i in range(sides):
        G.add_edge(ns[i], ns[(i + 1) % sides])

In [ ]:
# initialize graph
G = nx.Graph()

G.add_node(0, pos=0)
G.add_node(1, pos=1)
G.add_edge(0, 1)

# define figure
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
size = 10
ax.set_xlim([-size, size])
ax.set_ylim([-size, size])
fig.tight_layout()

positions = nx.get_node_attributes(G, "pos")
# convert complex position to a 2D tuple
positions = {key: (value.real, value.imag) for key, value in positions.items()}
nx.draw_networkx(G, positions, ax, with_labels=False, node_size=0, edge_color="white")

irregular = False


def onclick(event):
    global irregular

    click_pos = event.xdata + event.ydata * 1j
    positions = nx.get_node_attributes(G, "pos")
    positions = list(positions.items())
    positions.sort(key=lambda pair: abs(pair[1] - click_pos))
    n1, pos1 = positions[0]
    n2, pos2 = positions[1]

    # find on which side the click was
    side = ((click_pos - pos1) / (pos2 - pos1)).imag
    if 0 > side:
        # if it's the bad side, flip nodes
        n1, n2 = n2, n1

    if event.button == 1 and not event.dblclick:  # left click
        poly(3, G, n1, n2)
    elif event.button == 3 and not irregular:  # right click
        poly(4, G, n1, n2)
    elif event.button == 1 and event.dblclick:
        poly(5, G, n1, n2)
    elif event.button == 3 and irregular:
        poly(7, G, n1, n2)

    # convert complex position to a 2D tuple
    positions = nx.get_node_attributes(G, "pos")
    positions = {key: (value.real, value.imag) for key, value in positions.items()}
    nx.draw_networkx(
        G, positions, ax, with_labels=False, node_size=0, edge_color="white"
    )


cid = fig.canvas.mpl_connect("button_press_event", onclick)

In [ ]:
irregular = not irregular
irregular